In [1]:
!pip -q install langchain huggingface_hub python-dotenv

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### Basic LLMChain

In [3]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain

In [4]:
llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature": 0}
)

In [5]:
article = """ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 2022. It is notable for enabling users to refine and steer a conversation towards a desired length, format, style, level of detail, and language used. Successive prompts and replies are taken into account at each stage of the conversation as a context. While its content spans various domains of knowledge, ChatGPT has displayed a tendency to confidently provide inaccurate information.
By January 2023, it had become the fastest-growing consumer software application in history, gaining over 100 million users and contributing to OpenAI's valuation growing to US$29 billion. Within months, other businesses accelerated competing products such as Google's PaLM-E, Baidu's ERNIE, and Meta's LLaMA.[6] Some observers expressed concern over the potential of ChatGPT to displace or atrophy human intelligence, and its potential to enable plagiarism or fuel misinformation.
ChatGPT is built upon GPT-3.5 and GPT-4, from OpenAI's proprietary series of foundational GPT models. These large language models (LLMs) have been fine-tuned for conversational applications using a combination of supervised and reinforcement learning techniques. ChatGPT was released as a freely available research preview, but due to its popularity, OpenAI now operates the service on a freemium model. It allows users on its free tier to access the GPT-3.5 based version, while the more-advanced GPT-4 based version, as well as priority access to newer features, are provided to paid subscribers under the commercial name 'ChatGPT Plus'."""
len(article)

1603

In [6]:
fact_extraction_prompt = PromptTemplate(
    input_variables=["text_input"],
    template="Extract the key facts out of this text. Don't include opinions. Give each fact a number and keep them short sentences. \n\n{text_input}"
)

In [7]:
fact_extraction_chain = LLMChain(llm=llm, prompt=fact_extraction_prompt, verbose=False)

facts = fact_extraction_chain.run(article)

In [8]:
investor_update_prompt = PromptTemplate(
    input_variables=["facts"],
    template="You are a Goldman Sachs analyst. Take the following list of facts and use them to write a short paragrah for investors. Don't leave out key info:\n\n{facts}"
)

In [9]:
investor_update_chain = LLMChain(llm=llm, prompt=investor_update_prompt, verbose=True)

investor_update_chain.run(facts)



> Entering new LLMChain chain...
Prompt after formatting:
You are a Goldman Sachs analyst. Take the following list of facts and use them to write a short paragrah for investors. Don't leave out key info:

ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 20

> Finished chain.


'ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 20'

In [10]:
triples_prompt = PromptTemplate(
    input_variables=["facts"],
    template="Take the following list of facts and turn them into triples for a knowledge graph:\n\n {facts}"
)

In [11]:
triples_chain = LLMChain(llm=llm, prompt=triples_prompt, verbose=True)

triples = triples_chain.run(facts)
print(triples)
len(triples)



> Entering new LLMChain chain...
Prompt after formatting:
Take the following list of facts and turn them into triples for a knowledge graph:

 ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 20

> Finished chain.
ChatGPT ORIGINAL_NAMES "ChatGPT" ChatGPT


40

### Chaining these together

In [12]:
from langchain.chains import SimpleSequentialChain, SequentialChain

# A -> B -> C -> ...
full_chain = SimpleSequentialChain(chains=[fact_extraction_chain, investor_update_chain], verbose=True)

In [13]:
full_chain.run(article)



> Entering new SimpleSequentialChain chain...
ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 20


> Entering new LLMChain chain...
Prompt after formatting:
You are a Goldman Sachs analyst. Take the following list of facts and use them to write a short paragrah for investors. Don't leave out key info:

ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 20

> Finished chain.
ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 20

> Finished chain.


'ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 20'

### PAL Math Chain

In [14]:
from langchain.chains import PALChain

In [15]:
pal_chain = PALChain.from_math_prompt(llm, verbose=True)

In [16]:
question = "3 + 5 = ?"
pal_chain.run(question)



> Entering new PALChain chain...
def solution(): """3 + 5 = ? """ return 

> Finished chain.


'SyntaxError(\'invalid syntax\', (\'<string>\', 1, 34, \'def solution(): """3 + 5 = ? """ return \\n\', 1, 40))'

### API Chain

In [17]:
from langchain.chains.api.prompt import API_RESPONSE_PROMPT

from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate

In [18]:
from apis.food_list import API_DOCS
api_chain = APIChain.from_llm_and_api_docs(llm, API_DOCS, verbose=True)

In [19]:
api_chain.run("Please suggest some food for me")



> Entering new APIChain chain...
api_url= http://localhost:3000/v1/foods a='' b
http://localhost:3000/v1/foods a='' b
["Salmon","Xiao Long Bao","Twice Cooked Pork","Lanzhou noodles"]

> Finished chain.


"a='' b"